In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report



In [4]:
# Dataset path
data_dir = "dataset"
img_height = 128
img_width = 128
batch_size = 32

# Load train and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Check classes
class_names = train_ds.class_names
print("Classes:", class_names)


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [5]:
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Data Augmentation Block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Apply preprocessing
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

# Prefetch to improve performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Data augmentation and preprocessing done.")


✅ Data augmentation and preprocessing done.


In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Create datasets again without prefetch for now
train_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get class names BEFORE prefetching
class_names = train_ds_raw.class_names
num_classes = len(class_names)

# Extract all labels from dataset
train_labels = []
for _, labels in train_ds_raw.unbatch():
    train_labels.append(int(labels.numpy()))

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Print class weights nicely
for i, name in enumerate(class_names):
    print(f"{name}: {class_weights_dict[i]:.2f}")


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
cardboard: 1.05
glass: 0.85
metal: 1.02
paper: 0.72
plastic: 0.86
trash: 3.12


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class names BEFORE prefetching
class_names = train_ds_raw.class_names
num_classes = len(class_names)

# Extract all labels from training dataset
train_labels = []
for _, labels in train_ds_raw.unbatch():
    train_labels.append(int(labels.numpy()))

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert to dictionary
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Print class weights nicely
for i, name in enumerate(class_names):
    print(f"{name}: {class_weights_dict[i]:.2f}")


cardboard: 1.05
glass: 0.85
metal: 1.02
paper: 0.72
plastic: 0.86
trash: 3.12


In [9]:
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam

# Load the EfficientNetV2B2 base model
base_model = EfficientNetV2B2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

# Freeze the base model so it won't train initially
base_model.trainable = False

# Build the model
inputs = Input(shape=(128, 128, 3))
x = data_augmentation(inputs)  # Use the augmentation layer we created earlier
x = base_model(x, training=False)  # Pass through EfficientNet
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(6, activation='softmax')(x)  # 6 classes

model = models.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b2 (Functional)  │ (None, 4, 4, 1408)     │     8,769,374 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1408)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │         8,454 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,777,828 (33.48 MB)

 Trainable params: 8,454 (33.02 KB)

 Non-trainable params: 8,769,374 (33.45 MB)

In [11]:
# Save raw datasets before prefetching
train_ds_raw = train_ds
val_ds_raw = val_ds


In [12]:
# Train the model
history = model.fit(
    train_ds_raw,
    validation_data=val_ds_raw,
    epochs=10,
    class_weight=class_weights_dict
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.4061 - loss: 1.5200 - val_accuracy: 0.7505 - val_loss: 0.8171
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 54s 837ms/step - accuracy: 0.7199 - loss: 0.8469 - val_accuracy: 0.7822 - val_loss: 0.6655
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 48s 734ms/step - accuracy: 0.7491 - loss: 0.6928 - val_accuracy: 0.7941 - val_loss: 0.6109
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 48s 748ms/step - accuracy: 0.7939 - loss: 0.6259 - val_accuracy: 0.7980 - val_loss: 0.5645
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 50s 784ms/step - accuracy: 0.8019 - loss: 0.5728 - val_accuracy: 0.8040 - val_loss: 0.5464
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 48s 745ms/step - accuracy: 0.8199 - loss: 0.5348 - val_accuracy: 0.8139 - val_loss: 0.5270
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 45s 694ms/step - accuracy: 0.8233 - loss: 0.5187 - val_accuracy: 0.8277 - val_loss: 0.5033
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 669ms/step - accuracy: 0.8201 - loss: 0.4984 - val_accura

In [13]:
# ✅ Model training done
# ✅ Evaluation metrics computed
# ✅ Model ready for saving


In [14]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_ds_raw)
print(f"Validation Accuracy: {val_accuracy:.2f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8101 - loss: 0.4962
Validation Accuracy: 0.82


In [21]:
model.save("garbage_classifier_model_week2.keras")


In [22]:
from tensorflow.keras.models import load_model

# Try loading the saved model
model_loaded = load_model("garbage_classifier_model.keras")

print("Model loaded successfully ✅")

Model loaded successfully ✅
